In [1]:
import torch
import pandas as pd
import numpy as np
import transformers
from transformers import pipeline, AutoTokenizer,GPT2Tokenizer, GPT2Model, AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
import os

2024-03-15 22:16:07.633379: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
rewrite_data_df = pd.read_csv('data/8000_data.csv')
rewrite_data_df = rewrite_data_df.dropna()
rewrite_data_df.head()

,original_text,prompt,rewritten_text
0,I get it now. You have to give to get. Where's...,Convey the same message as this text but throu...,"""The cosmic tapestry of existence unraveled be..."
1,Atlas was not seen to be special by any means....,Restyle this text as if it were written by a A...,His emerald glow illuminated the celestial can...
2,Gleaming eyes shining in the dark. Visions of...,Adapt this text as a script for a wizard in a ...,(A smoky bar in the heart of a roaring twentie...
3,"Remember, there's security cameras at every co...",Convey the same message as this text but throu...,"""My dear reader, I have come from the future t..."
4,Waves 17 Septillion ships per cloud Like loc...,Imagine this text was a villain in the world o...,If written as a villain in the world of villai...


## Initial Model

In [3]:
from transformers import BertModel, GPT2Model, GPT2Config, PreTrainedModel, BertTokenizer, GPT2Tokenizer
import torch
import torch.nn.functional as F

class BertGPT2PromptModel(PreTrainedModel):
    def __init__(self, encoder_pretrained_model_name='bert-base-uncased', decoder_pretrained_model_name='gpt2'):
        super(BertGPT2PromptModel, self).__init__(GPT2Config.from_pretrained(decoder_pretrained_model_name))
        
        self.encoder = BertModel.from_pretrained(encoder_pretrained_model_name)
        self.decoder = GPT2Model.from_pretrained(decoder_pretrained_model_name)
        
        assert self.encoder.config.hidden_size == self.decoder.config.n_embd, "Encoder and decoder hidden sizes do not match"
        
    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, use_teacher_forcing=False):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        encoder_last_hidden_state = encoder_outputs.last_hidden_state

        # Apply teacher forcing if enabled and decoder_input_ids are provided
        if use_teacher_forcing and decoder_input_ids is not None:
            # Here, we pass decoder_input_ids directly as input to the decoder
            decoder_outputs = self.decoder(input_ids=decoder_input_ids, attention_mask=decoder_attention_mask)
        else:
            # If not using teacher forcing, use the encoded states as inputs_embeds to the decoder
            decoder_outputs = self.decoder(inputs_embeds=encoder_last_hidden_state, attention_mask=decoder_attention_mask)
        
        return decoder_outputs


In [4]:
gpt_prompt = lambda inp, out: f"""You are an intelligent prompt generator. I will provide you with an input and a llm-generated output. Your task is to identify the prompt that instructed an LLM to rewrite the input into the output. Please refer to the example below for the desired format.

Input: "This like betting your life savings on a game of black jack, win once, and then walk. Brilliant!"
Output: "\n\n\Good evening, and welcome to the future, folks. Tonight, we bring you a story that will have you on the edge of your seat. It\'s a tale of a game that\'s been captivating the world, and it\'s one that you won\'t believe is actually happening."
Prompt: Narrate this as if it were a news report from future

Input: "The security sensors work on *sound*, though, not light. Regardless of what disguise you use to enter, they can catch you. "
Output: "Imagine you're playing hide and seek, but instead of using your eyes to find someone, you use your ears to listen for sounds they make. The security sensors are like that; they listen for sounds to catch people, not look for them with eyes. So, even if you dress up differently, if you make a sound, they can still find you!"
Prompt: Explain this to me like I\'m five
        
Input: "{inp}"
Output: "{out}"
Prompt:""".lower()


In [5]:
encoder_model_name = 'bert-base-uncased'
decoder_model_name = 'gpt2'

In [6]:
model = BertGPT2PromptModel(encoder_pretrained_model_name=encoder_model_name, decoder_pretrained_model_name=decoder_model_name)
tokenizer_encoder = BertTokenizer.from_pretrained(encoder_model_name)
tokenizer_decoder = GPT2Tokenizer.from_pretrained(decoder_model_name)

test_input = "This like betting your life savings on a game of black jack, win once, and then walk. Brilliant!"
test_output = "\n\n(Verse 1)\nGather 'round me, me hearties, and listen to a tale\nOf a game of black jack, where fate's a wail\nWith a full hand's worth of savings, you're ready to play\nSo stake your dough upon the table, and dance to the bay.\n\n(Chorus)\nThis like betting on black jack, a game of high stakes\nWin once and then walk"

# Create gpt_prompt
test_prompt = gpt_prompt(test_input, test_output)

# test_prompt = gpt_prompt(rewrite_data_df.iloc[0][0], rewrite_data_df.iloc[0][2])

# Tokenize input 
input_ids = tokenizer_encoder(test_prompt, return_tensors='pt').input_ids

# Forward pass through the model
with torch.no_grad():
    outputs = model(input_ids)
    
output_probs = outputs[0].argmax(-1)
prompt_text = tokenizer_decoder.decode(output_probs.squeeze().tolist(), skip_special_tokens=True) #not predicting spaces
prompt_text

In [7]:
output_probs = outputs[0].argmax(-1)
prompt_text = tokenizer_decoder.decode(output_probs.squeeze().tolist(), skip_special_tokens=True) #not predicting spaces
prompt_text

'ageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageage

# Load Data

In [9]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, GPT2Tokenizer

class TextDataset(Dataset):
    def __init__(self, dataframe, bert_tokenizer, gpt2_tokenizer, max_length=512):
        self.dataframe = dataframe
        self.max_length = max_length

        if gpt2_tokenizer.pad_token is None:
            gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

        self.bert_encoded = bert_tokenizer.batch_encode_plus(
            dataframe['input'].tolist(),
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )

        self.gpt2_encoded = gpt2_tokenizer.batch_encode_plus(
            dataframe['prompt'].tolist(),
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        input_ids = self.bert_encoded['input_ids'][idx]
        attention_mask = self.bert_encoded['attention_mask'][idx]

        # For GPT-2, we create decoder_input_ids which is used for teacher forcing during training
        decoder_input_ids = self.gpt2_encoded['input_ids'][idx][:-1]
        labels = self.gpt2_encoded['input_ids'][idx][1:].clone().detach()

        # Replace padding token id's of the labels by -100 so that they are not considered in the loss
        labels[labels == gpt2_tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'decoder_input_ids': decoder_input_ids,
            'labels': labels
        }



In [66]:
# first, use simple connecting. 

df = pd.read_csv("data/8000_data.csv")
df = df.dropna()
df['input'] = df['original_text'] + ' ' + df['rewritten_text']

In [71]:
X_train, X_test, y_train, y_test = train_test_split(df[['input']], df['prompt'], test_size=0.33, random_state=42)
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

In [72]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

if bert_tokenizer.pad_token is None:
    bert_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# bert_tokenizer.pad_token = bert_tokenizer.eos_token
# gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

train_dataset = TextDataset(train_data, bert_tokenizer, gpt2_tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

test_dataset = TextDataset(test_data, bert_tokenizer, gpt2_tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=True)


In [73]:
from transformers import BertModel, BertConfig
import torch

class BertModelModified(BertModel):
    def __init__(self, config):
        super().__init__(config)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, output_attentions=None, output_hidden_states=None, return_dict=None):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        outputs = super().forward(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=True,  # Ensure that hidden states are returned
            return_dict=return_dict,
        )

        # Accessing the second-to-last hidden state, which are the inputs to the last layer
        second_to_last_hidden_states = outputs.hidden_states[-2]

        # You can modify this return statement based on what you need
        # For instance, you might want to return just the last hidden state and the second-to-last hidden states
        return outputs.last_hidden_state, second_to_last_hidden_states

# Example usage
config = BertConfig()
bert_model_modified = BertModelModified(config)

In [49]:
BERT_LABEL_LIMIT = 768
REPLACE_TOKEN = 50

In [12]:
encoder = BertModel.from_pretrained('bert-base-uncased')
# encoder = bert_model_modified #__main__.BertModelModified
isinstance(encoder, transformers.models.bert.modeling_bert.BertModel)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

True

In [57]:
from transformers import BertModel, GPT2Model, GPT2Config, PreTrainedModel
import torch
import torch.nn as nn

class BertGPT2PromptModel(PreTrainedModel):
    def __init__(self, encoder, decoder_pretrained_model_name='gpt2'):
        decoder_config = GPT2Config.from_pretrained(decoder_pretrained_model_name, add_cross_attention=True)
        super(BertGPT2PromptModel, self).__init__(decoder_config)
        
        self.encoder = encoder
        self.decoder = GPT2Model.from_pretrained(decoder_pretrained_model_name, config=decoder_config)
        
        assert self.encoder.config.hidden_size == self.decoder.config.n_embd, "Encoder and decoder hidden sizes do not match"
        
        # max pooling layer, trying to copy Bert's max pooling layer
        self.global_max_pool = nn.AdaptiveMaxPool1d(self.decoder.config.n_embd) 
        
        # fully connected layer to recover dimension
        self.fc = nn.Linear(self.encoder.config.hidden_size, self.decoder.config.n_embd)
        self.start_token_id = self.decoder.config.eos_token_id
        
    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, encoder_hidden_states=None, use_teacher_forcing=False):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        
        if isinstance(self.encoder, BertModelModified):
            _, second_to_last_hidden_states = encoder_outputs  
            
            # Apply global max pooling and the fully connected layer
            #print(second_to_last_hidden_states.shape) # [2, 512, 768]
            pooled_output = self.global_max_pool(second_to_last_hidden_states)#.squeeze(-1)  # Change dimensions for pooling
            #print(pooled_output.shape) # [2, 512, 1]
            
            encoder_hidden_states = self.fc(pooled_output)
            
            
        if use_teacher_forcing and decoder_input_ids is not None:
            start_tokens = torch.ones((input_ids.size(0), 1), dtype=torch.long, device=input_ids.device) * self.start_token_id
            decoder_input_ids = torch.cat((start_tokens, decoder_input_ids), dim=1)

            decoder_outputs = self.decoder(
                input_ids=decoder_input_ids, 
                attention_mask=decoder_attention_mask,
                encoder_hidden_states=encoder_hidden_states, # [2, 512, 1].view(-1, x.size(-1)) -> [1024, 1]
                use_cache=False  # Turn off caching when using cross-attention
            )
        else:
            # Initialize the list to store decoder outputs
            decoder_output_sequences = []
            max_length = 100

            # Start token for the first input to the decoder
            start_tokens = torch.ones((input_ids.size(0), 1), dtype=torch.long, device=input_ids.device) * self.start_token_id
            decoder_input = start_tokens

            # Loop to generate sequence
            for _ in range(max_length):  # max_length is a predefined maximum length for generation
                decoder_outputs = self.decoder(
                    input_ids=decoder_input, 
                    attention_mask=decoder_attention_mask,
                    encoder_hidden_states=encoder_hidden_states,
                    use_cache=False  # Turn off caching when using cross-attention
                )

                # Get the last token in the sequence
                next_token_logits = decoder_outputs.last_hidden_state[:, -1, :]
                next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)

                # Append generated token to the list
                decoder_output_sequences.append(next_token)

                # Prepare the next input token
                decoder_input = torch.cat((decoder_input, next_token), dim=1)

            # Concatenate the list of generated tokens to form the final sequence
            decoder_outputs = torch.cat(decoder_output_sequences, dim=1)
        
        return decoder_outputs


In [74]:
import torch.nn.functional as F

model = BertGPT2PromptModel(encoder)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

torch.cuda.empty_cache()
model.to(device)

total_loss = 0
epochs = 1
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}", unit="batch"):

        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        decoder_input_ids = batch['decoder_input_ids'].to(device)
        labels = batch['labels'].to(device)
        
        # this is where the problem is
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, use_teacher_forcing=True)

        logits = outputs.last_hidden_state
        logits = logits[:, :-1, :]

        logits = logits.reshape(-1, logits.size(-1))
        labels = labels.view(-1)
        
        
        labels[labels > BERT_LABEL_LIMIT] = REPLACE_TOKEN
        
        # Calculate the loss
        loss = F.cross_entropy(logits, labels, ignore_index=-100) 

        loss.backward()
        total_loss += loss.item()
        
#         for name, param in model.named_parameters():
#             if param.requires_grad:
#                 if param.grad is None:
#                     print(name)
# #                 if torch.any(torch.isnan(param.grad)) or torch.any(torch.isinf(param.grad)):
# #                     print(f"NaN or Inf found in gradients of {name}")

        optimizer.step()
    
    print(f"Epoch {epoch + 1}/{epochs} | Loss: {total_loss/len(dataloader)}")


Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.crossattention.c_attn.bias', 'h.0.crossattention.c_attn.weight', 'h.0.crossattention.c_proj.bias', 'h.0.crossattention.c_proj.weight', 'h.0.crossattention.q_attn.bias', 'h.0.crossattention.q_attn.weight', 'h.0.ln_cross_attn.bias', 'h.0.ln_cross_attn.weight', 'h.1.crossattention.c_attn.bias', 'h.1.crossattention.c_attn.weight', 'h.1.crossattention.c_proj.bias', 'h.1.crossattention.c_proj.weight', 'h.1.crossattention.q_attn.bias', 'h.1.crossattention.q_attn.weight', 'h.1.ln_cross_attn.bias', 'h.1.ln_cross_attn.weight', 'h.10.crossattention.c_attn.bias', 'h.10.crossattention.c_attn.weight', 'h.10.crossattention.c_proj.bias', 'h.10.crossattention.c_proj.weight', 'h.10.crossattention.q_attn.bias', 'h.10.crossattention.q_attn.weight', 'h.10.ln_cross_attn.bias', 'h.10.ln_cross_attn.weight', 'h.11.crossattention.c_attn.bias', 'h.11.crossattention.c_attn.weight', 'h.11.crossattentio

KeyboardInterrupt: 

### Test one example

In [63]:
# model = BertGPT2PromptModel(encoder_pretrained_model_name=encoder_model_name, decoder_pretrained_model_name=decoder_model_name)
tokenizer_encoder = BertTokenizer.from_pretrained(encoder_model_name)
tokenizer_decoder = GPT2Tokenizer.from_pretrained(decoder_model_name)

# test_input = "This like betting your life savings on a game of black jack, win once, and then walk. Brilliant!"
# test_output = "\n\n(Verse 1)\nGather 'round me, me hearties, and listen to a tale\nOf a game of black jack, where fate's a wail\nWith a full hand's worth of savings, you're ready to play\nSo stake your dough upon the table, and dance to the bay.\n\n(Chorus)\nThis like betting on black jack, a game of high stakes\nWin once and then walk"

# # Create gpt_prompt
# test_prompt = gpt_prompt(test_input, test_output)

# # test_prompt = gpt_prompt(rewrite_data_df.iloc[0][0], rewrite_data_df.iloc[0][2])

# # Tokenize input 
# input_ids = tokenizer_encoder(test_prompt, return_tensors='pt').input_ids

model.eval()
# Forward pass through the model
with torch.no_grad():
    outputs = model(input_ids, use_teacher_forcing=True)
    
output_probs = outputs[0]#.argmax(-1)
prompt_text = tokenizer_decoder.decode(output_probs.squeeze().tolist(), skip_special_tokens=True) #not predicting spaces
prompt_text

'ageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageageraage Iageageage Iage I I I rageageageageageageageageraage I rageraraage I rraage I rrararararaage r I rage I rrararararararara'

### Test for sharp cosine similarity

In [65]:
def batch_cosine_similarity(x1, x2):
    x1_norm = torch.nn.functional.normalize(x1, p=2, dim=-1)
    x2_norm = torch.nn.functional.normalize(x2, p=2, dim=-1)
    cos_sim = torch.mm(x1_norm, x2_norm.transpose(0, 1))
    return cos_sim

In [ ]:
model.eval()

total_test_loss = 0
predictions = []

with torch.no_grad():
    for batch in test_dataloader:  # Assuming you have a DataLoader for your test data
        
        # Transfer the batch data to the same device as your model
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        decoder_input_ids = batch['decoder_input_ids'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass: compute the model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, use_teacher_forcing=False)
        
        # Extract the logits
        logits = outputs.last_hidden_state
        logits = logits[:, :-1, :].reshape(-1, logits.size(-1))
        labels = labels.view(-1)
        
        labels[labels > BERT_LABEL_LIMIT] = REPLACE_TOKEN  # Use the same token replacement as in training
        
        loss = F.cross_entropy(logits, labels, ignore_index=-100)
        total_test_loss += loss.item()
        
        output_probs = outputs[0]#.argmax(-1)
        prompt_text = tokenizer_decoder.decode(output_probs.squeeze().tolist(), skip_special_tokens=True) #not predicting spaces
        
        #TODO: calculate cosine similarity between prompt_text and actual prompt

# Compute average test loss
average_test_loss = total_test_loss / len(test_dataloader)
print(f"Average test loss: {average_test_loss}")

# Process your predictions as needed
# For example, you might want to aggregate batch predictions, convert token IDs back to text, etc.


### experiments


In [ ]:
# [name for name, param in model.named_parameters()]
torch.any(torch.isnan(labels))

In [ ]:
# ~0.9-2 percent of labels are larger than 768
exceed = np.sum([int(ele) for ele in (labels.sort(descending=True).values > 768)])
print(exceed)

data = logits.unique().detach().numpy()

plt.hist(data, bins=100, alpha=0.6, color='g')
plt.title('Distribution of Tensor Values')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

sum(data == 50)